In [1]:
%load_ext autoreload
%autoreload 2

In [67]:
import warnings
warnings.filterwarnings("ignore")

# deepOF model evaluation

Given a dataset and a trained model, this notebook allows the user to 

* Load and inspect the different models (encoder, decoder, grouper, gmvaep)
* Visualize reconstruction quality for a given model
* Visualize a static latent space
* Visualize trajectories on the latent space for a given video
* sample from the latent space distributions and generate video clips showcasing generated data

In [2]:
import os
os.chdir(os.path.dirname("../"))

In [15]:
import deepof.data
import deepof.utils
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

from ipywidgets import interact
from IPython import display
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import interact

### 1. Define and run project

In [75]:
path = os.path.join("..", "..", "Desktop", "deepoftesttemp")
trained_network = os.path.join("..", "..", "Desktop")
exclude_bodyparts = ["Tail_1", "Tail_2", "Tail_tip", "Tail_base"]
window_size = 24

In [54]:
%%time
proj = deepof.data.project(
    path=path,
    smooth_alpha=0.99,
    exclude_bodyparts=exclude_bodyparts,
    arena_dims=[380],
)

CPU times: user 280 ms, sys: 22.3 ms, total: 303 ms
Wall time: 250 ms


In [55]:
%%time
proj = proj.run(verbose=True)
print(proj)

Loading trajectories...
Smoothing trajectories...
Interpolating outliers...
Iterative imputation of ocluded bodyparts...
Computing distances...
Computing angles...
Done!
deepof analysis of 2 videos
CPU times: user 2.63 s, sys: 85.8 ms, total: 2.71 s
Wall time: 634 ms


### 2. Load pretrained deepof model

In [56]:
coords = proj.get_coords(center="Center", align="Spine_1", align_inplace=True)
preprocessed_data,_,_,_ = coords.preprocess(test_videos=0, window_step=5, window_size=window_size)

In [57]:
# Set model parameters
encoding=6
loss="ELBO"
k=25
pheno=0
predictor=0

In [58]:
encoder, decoder, grouper, gmvaep = deepof.models.SEQ_2_SEQ_GMVAE(
    loss=loss,
    number_of_components=k,
    compile_model=True,
    encoding=encoding,
    predictor=predictor,
    phenotype_prediction=pheno,
).build(preprocessed_data.shape)[:4]

gmvaep.load_weights(
    os.path.join(
        trained_network, [i for i in os.listdir(trained_network) if i.endswith("h5")][0]
    )
)

ValueError: You are trying to load a weight file containing 15 layers into a model with 14 layers.

In [59]:
# Uncomment to see model summaries
# encoder.summary()
# decoder.summary()
# grouper.summary()
# gmvaep.summary()

In [60]:
# Uncomment to plot model structure
def plot_model(model, name):
    tf.keras.utils.plot_model(
        model,
        to_file=os.path.join(
            path,
            "deepof_{}_{}.png".format(name, datetime.now().strftime("%Y%m%d-%H%M%S")),
        ),
        show_shapes=True,
        show_dtype=False,
        show_layer_names=True,
        rankdir="TB",
        expand_nested=True,
        dpi=200,
    )


# plot_model(encoder, "encoder")
# plot_model(decoder, "decoder")
# plot_model(grouper, "grouper")
# plot_model(gmvaep, "gmvaep")

### 4. Pass data through all models

In [61]:
encodings = encoder.predict(preprocessed_data)
groupings = grouper.predict(preprocessed_data)
reconstrs = gmvaep.predict(preprocessed_data)

### 5. Evaluate reconstruction (to be incorporated into deepof.evaluate)

In [62]:
# Fit a scaler to the data, to back-transform reconstructions later
scaler = StandardScaler().fit(preprocessed_data[:, 0, :])

In [63]:
# Rescale reconstructions
rescaled_reconstructions = scaler.inverse_transform(
    reconstrs.reshape(reconstrs.shape[0] * reconstrs.shape[1], reconstrs.shape[2])
)
rescaled_reconstructions = rescaled_reconstructions.reshape(reconstrs.shape)

In [98]:
# Auxiliary animation functions


def plot_mouse_graph(instant_x, instant_y, instant_rec_x, instant_rec_y, ax, edges):
    """Generates a graph plot of the mouse"""
    plots = []
    rec_plots = []
    for edge in edges:
        (temp_plot,) = ax.plot(
            [float(instant_x[edge[0]]), float(instant_x[edge[1]])],
            [float(instant_y[edge[0]]), float(instant_y[edge[1]])],
            color="#006699",
        )
        (temp_rec_plot,) = ax.plot(
            [float(instant_rec_x[edge[0]]), float(instant_rec_x[edge[1]])],
            [float(instant_rec_y[edge[0]]), float(instant_rec_y[edge[1]])],
            color="#006699",
        )
        plots.append(temp_plot)
        rec_plots.append(temp_rec_plot)
    return plots


def update_mouse_graph(x, y, plots, edges):
    """Updates the graph plot to enable animation"""

    for plot, edge in zip(plots, edges):
        plot.set_data(
            [float(x[edge[0]]), float(x[edge[1]])],
            [float(y[edge[0]]), float(y[edge[1]])],
        )

In [100]:
# Display a video with the original data superimposed with the reconstructions

random_exp = np.random.choice(list(coords.keys()), 1)[0]


@interact(time_slider=(0.0, 15000, 500), length_slider=(0, 1000, 100))
def animate_mice_across_time(time_slider, length_slider):

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    edges = deepof.utils.connect_mouse_topview()

    for bpart in exclude_bodyparts:
        edges.remove_node(bpart)

    for limb in ["Left_fhip", "Right_fhip", "Left_bhip", "Right_bhip"]:
        edges.remove_edge("Center", limb)

    edges = edges.edges()

    data = coords[random_exp].loc[time_slider : time_slider + length_slider - 1, :]
    data_rec = gmvaep.predict(
        coords.filter_videos([random_exp]).preprocess(
            test_videos=0, window_step=5, window_size=window_size
        )[0]
    )
    data_rec = pd.DataFrame(scaler.inverse_transform(data_rec[:, 24 // 2, :]))
    data_rec.columns = data.columns

    data["Center", "x"] = 0
    data["Center", "y"] = 0
    data_rec["Center", "x"] = 0
    data_rec["Center", "y"] = 0

    init_x = data.xs("x", level=1, axis=1, drop_level=False).iloc[0, :]
    init_y = data.xs("y", level=1, axis=1, drop_level=False).iloc[0, :]
    init_rec_x = data_rec.xs("x", level=1, axis=1, drop_level=False).iloc[0, :]
    init_rec_y = data_rec.xs("y", level=1, axis=1, drop_level=False).iloc[0, :]

    plots = plot_mouse_graph(init_x, init_y, init_rec_x, init_rec_y, ax, edges)
    scatter = ax.scatter(x=np.array(init_x), y=np.array(init_y), color="#006699",)
    rec_scatter = ax.scatter(
        x=np.array(init_rec_x), y=np.array(init_rec_y), color="#006699",
    )

    # Update data in main plot
    def animation_frame(i):
        # Update scatter plot
        x = data.xs("x", level=1, axis=1, drop_level=False).iloc[i, :]
        y = data.xs("y", level=1, axis=1, drop_level=False).iloc[i, :]
        rec_x = data_rec.xs("x", level=1, axis=1, drop_level=False).iloc[i, :]
        rec_y = data_rec.xs("y", level=1, axis=1, drop_level=False).iloc[i, :]
        
        scatter.set_offsets(np.c_[np.array(x), np.array(y)])
        scatter.set_offsets(np.c_[np.array(rec_x), np.array(rec_y)])
        update_mouse_graph(x, y, plots, edges)

        return scatter

    animation = FuncAnimation(
        fig, func=animation_frame, frames=length_slider, interval=100,
    )

    ax.set_title("Positions across time for centered data")
    ax.set_ylim(-100, 60)
    ax.set_xlim(-60, 60)
    ax.set_xlabel("x")
    ax.set_ylabel("y")

    video = animation.to_html5_video()
    html = display.HTML(video)
    display.display(html)
    plt.close()

interactive(children=(FloatSlider(value=7500.0, description='time_slider', max=15000.0, step=500.0), IntSlider…

### 6. Evaluate latent space (to be incorporated into deepof.evaluate)